In [1]:
%pip install torch transformers
%pip install fugashi ipadic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import BertForMaskedLM, BertJapaneseTokenizer
import torch
import csv
import json

c:\WorkSpace\Practice\env2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#model_name = "cl-tohoku/bert-base-japanese"
#model_name = "cl-tohoku/bert-base-japanese-v3"
#model_name = "cl-tohoku/bert-base-japanese-char-v3"
model_name = "cl-tohoku/bert-large-japanese-v2"
#model_name = "cl-tohoku/bert-large-japanese-char-v2"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

c:\WorkSpace\Practice\env2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cl-tohoku/bert-large-japanese-v2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def tokenize_with_mask(text, tokenizer):
    tokens = tokenizer.tokenize(text)
    masked_tokens = []
    for i in range(len(tokens)):
        if tokens[i] not in ['[CLS]', '[SEP]']:
            masked_tokens.append(tokens[:i] + ['[MASK]'] + tokens[i+1:])
    return masked_tokens

def calculate_score(masked_tokens, tokenizer, model):
    scores = []
    for tokens in masked_tokens:
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        tensor_input = torch.tensor([input_ids])
        with torch.no_grad():
            outputs = model(tensor_input, labels=tensor_input)
            loss = outputs[0]
            scores.append(loss.item())
    return sum(scores) / len(scores)

In [25]:
#テキストの読み込み
with open("E:\実験\実験結果/18.文章25\文章25.txt", 'r', encoding='utf-8') as file:
    texts = file.readlines()

results = []
for text in texts[:100]: 
    text = text.strip()
    masked_tokens = tokenize_with_mask(text, tokenizer)
    score = calculate_score(masked_tokens, tokenizer, model)
    results.append({'text': text, 'score': score})

In [26]:
# CSVファイルに保存
with open('18.25.csv', 'w', encoding='shift-JIS',newline='', errors='replace') as csv_file:
    fieldnames = ['text', 'score']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

In [22]:
# CSVファイルに保存
with open('4.2.csv', 'w', encoding='utf-8', newline='') as csv_file:
    fieldnames = ['text', 'score']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

PermissionError: [Errno 13] Permission denied: '4.2.csv'